<h2>This Notebook is to show how I use various open-source modules to build models, implement ML techniques on said models, and predict unknown data points.</h2>

In [24]:
import pybaseball as pyb
import pandas as pd
import numpy as np
import seaborn as sns

Since we will have large pd.DataFrames, we will set the display options to show all rows and columns

In [25]:
pd.options.display.max_rows=None
pd.options.display.max_columns=None

now to get our teams data from pybaseball using the teams() call, we will save this to a variable 'team_data'

In [26]:
team_data = pyb.teams()
print(team_data)

      yearID lgID teamID franchID divID  Rank    G  Ghome    W    L DivWin  \
0       1871  NaN    BS1      BNA   NaN     3   31    NaN   20   10    NaN   
1       1871  NaN    CH1      CNA   NaN     2   28    NaN   19    9    NaN   
2       1871  NaN    CL1      CFC   NaN     8   29    NaN   10   19    NaN   
3       1871  NaN    FW1      KEK   NaN     7   19    NaN    7   12    NaN   
4       1871  NaN    NY2      NNA   NaN     5   33    NaN   16   17    NaN   
5       1871  NaN    PH1      PNA   NaN     1   28    NaN   21    7    NaN   
6       1871  NaN    RC1      ROK   NaN     9   25    NaN    4   21    NaN   
7       1871  NaN    TRO      TRO   NaN     6   29    NaN   13   15    NaN   
8       1871  NaN    WS3      OLY   NaN     4   32    NaN   15   15    NaN   
9       1872  NaN    BL1      BLC   NaN     2   58    NaN   35   19    NaN   
10      1872  NaN    BR1      ECK   NaN     9   29    NaN    3   26    NaN   
11      1872  NaN    BR2      BRA   NaN     6   37    NaN    9  

<h4> As you can see our DataFrame has team statistics dating all the way back the 1871. Baseball in 1871 compared to baseball that we know in 2021, are two vastly different games, not to mention the Boston BeanEaters are no longer an active orginazation in the MLB. So to get relevent teams and statistical info we will use the >= operators to sort the column['yearID'] to return all stats from 1961 forward (this is when MLB decided to bump a season up 8 games from 154, to 162) </h4>

In [33]:
team_data = team_data[team_data['yearID']>=1961]
team_data.head()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
1359,1961,AL,BAL,BAL,NaN,3,163,82.0,95,67,NaN,NaN,N,N,691,5481,1393,227,36,149,581.0,902.0,39.0,30.0,NaN,NaN,588,526,3.22,54,21,33,4413,1226,109,617,926,126,173,0.980,Baltimore Orioles,Memorial Stadium,951089.0,96,96,BAL,BAL,BAL
1360,1961,AL,BOS,BOS,NaN,6,163,82.0,76,86,NaN,NaN,N,N,729,5508,1401,251,37,112,647.0,847.0,56.0,36.0,NaN,NaN,792,687,4.29,35,6,30,4326,1472,167,679,831,143,140,0.977,Boston Red Sox,Fenway Park II,850589.0,102,103,BOS,BOS,BOS
1361,1961,AL,CHA,CHW,NaN,4,163,81.0,86,76,NaN,NaN,N,N,765,5556,1475,216,46,138,550.0,612.0,100.0,40.0,NaN,NaN,726,653,4.06,39,3,33,4344,1491,158,498,814,128,138,0.980,Chicago White Sox,Comiskey Park,1146019.0,99,97,CHW,CHA,CHA
1362,1961,NL,CHN,CHC,NaN,7,156,78.0,64,90,NaN,NaN,N,N,689,5344,1364,238,51,176,539.0,1027.0,35.0,25.0,NaN,NaN,800,689,4.48,34,6,25,4155,1492,165,465,755,183,175,0.970,Chicago Cubs,Wrigley Field,673057.0,101,104,CHC,CHN,CHN
1363,1961,NL,CIN,CIN,NaN,1,154,77.0,93,61,NaN,NaN,Y,N,710,5243,1414,247,35,158,423.0,761.0,70.0,33.0,NaN,NaN,653,575,3.78,46,12,40,4110,1300,147,500,829,134,124,0.977,Cincinnati Reds,Crosley Field,1117603.0,102,101,CIN,CIN,CIN


now we will use the select the offensive and defensive categories into seperate DataFrames, but first we will create a run_diff dataframe

In [34]:
team_off = team_data[['yearID','lgID','franchID','Rank','G','W','L','R','AB','H','2B','3B','HR','BB','SO','SB','CS','HBP','SF']]
team_pitch = team_data[['yearID','lgID','franchID','Rank','G','W','L','RA','ER','ERA','CG','SHO','SV','IPouts','HA','HRA','BBA','SOA','E','DP','FP']]

In [35]:
team_off.head()
team_pitch.head()

,yearID,lgID,franchID,Rank,G,W,L,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP
1359,1961,AL,BAL,3,163,95,67,588,526,3.22,54,21,33,4413,1226,109,617,926,126,173,0.980
1360,1961,AL,BOS,6,163,76,86,792,687,4.29,35,6,30,4326,1472,167,679,831,143,140,0.977
1361,1961,AL,CHW,4,163,86,76,726,653,4.06,39,3,33,4344,1491,158,498,814,128,138,0.980
1362,1961,NL,CHC,7,156,64,90,800,689,4.48,34,6,25,4155,1492,165,465,755,183,175,0.970
1363,1961,NL,CIN,1,154,93,61,653,575,3.78,46,12,40,4110,1300,147,500,829,134,124,0.977


<h4>Now we will use the .corr() method to show us correlation of the stats ( we will be focusing on the 'R' column for offensive stats & 'RA' column for pitching stats. This is to see what offensive and defensive stats correlate most closely) they will be assigned to variables to show they are correllation sets, off_corr & pitch_corr respectively.</h4>

In [36]:
off_corr = team_off.corr()
print(off_corr)

          yearID      Rank         G         W         L         R        AB  \
yearID  1.000000 -0.286345 -0.144984 -0.082012 -0.082265  0.163446 -0.113256   
Rank   -0.286345  1.000000  0.046546 -0.624345  0.678456 -0.388174  0.012667   
G      -0.144984  0.046546  1.000000  0.590888  0.588000  0.621893  0.989730   
W      -0.082012 -0.624345  0.590888  1.000000 -0.304808  0.691412  0.606894   
L      -0.082265  0.678456  0.588000 -0.304808  1.000000  0.042843  0.560034   
R       0.163446 -0.388174  0.621893  0.691412  0.042843  1.000000  0.686864   
AB     -0.113256  0.012667  0.989730  0.606894  0.560034  0.686864  1.000000   
H      -0.034647 -0.160624  0.863856  0.661480  0.357445  0.851928  0.915165   
2B      0.472687 -0.275240  0.552982  0.460234  0.195614  0.777882  0.613957   
3B     -0.391241  0.057780  0.339077  0.231318  0.165784  0.186208  0.348197   
HR      0.409282 -0.324387  0.326563  0.429583 -0.042555  0.739687  0.373848   
BB     -0.086191 -0.225961  0.599286  0.

In [37]:
pitch_corr = team_pitch.corr()
print(pitch_corr)

          yearID      Rank         G         W         L        RA        ER  \
yearID  1.000000 -0.286345 -0.144984 -0.082012 -0.082265  0.160518  0.252141   
Rank   -0.286345  1.000000  0.046546 -0.624345  0.678456  0.284191  0.236119   
G      -0.144984  0.046546  1.000000  0.590888  0.588000  0.607969  0.578856   
W      -0.082012 -0.624345  0.590888  1.000000 -0.304808  0.003990  0.010743   
L      -0.082265  0.678456  0.588000 -0.304808  1.000000  0.715753  0.675228   
RA      0.160518  0.284191  0.607969  0.003990  0.715753  1.000000  0.988539   
ER      0.252141  0.236119  0.578856  0.010743  0.675228  0.988539  1.000000   
ERA     0.422948  0.262194 -0.089034 -0.462757  0.362143  0.716869  0.752761   
CG     -0.846306  0.132735  0.142142  0.173148 -0.011229 -0.242640 -0.318100   
SHO    -0.272329 -0.223489  0.313702  0.516175 -0.148907 -0.345459 -0.361572   
SV      0.318860 -0.445100  0.402975  0.609022 -0.131879  0.141843  0.179162   
IPouts -0.164320  0.021184  0.995809  0.